# Model

## Libs

In [ ]:
import gc
import json
import pandas as pd
import unicodedata
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## Data

In [ ]:
Data_path = '../data/json/'

In [ ]:
with open(Data_path + 'applicants.json', 'r', encoding='utf-8') as file:
	applicants = json.load(file)

rows = []
for applicant in applicants:
	id = applicant
	informacoes_pessoais = applicants.get(id).get("informacoes_pessoais")
	informacoes_profissionais = applicants.get(id).get("informacoes_profissionais")
	formacao_e_idiomas = applicants.get(id).get("formacao_e_idiomas")
	aux = {
		'id': int(id),
		'pcd': informacoes_pessoais.get('pcd'),
		'endereco': informacoes_pessoais.get('endereco'),
		'titulo_profissional': informacoes_profissionais.get('titulo_profissional'),
		'area_atuacao': informacoes_profissionais.get('area_atuacao'),
		'conhecimentos_tecnicos': informacoes_profissionais.get('conhecimentos_tecnicos'),
		'certificacoes': informacoes_profissionais.get('certificacoes'),
		'outras_certificacoes': informacoes_profissionais.get('outras_certificacoes'),
		'nivel_profissional': informacoes_profissionais.get('nivel_profissional'),
		'nivel_academico': formacao_e_idiomas.get('nivel_academico'),
		'nivel_ingles': formacao_e_idiomas.get('nivel_ingles'),
		'nivel_espanhol': formacao_e_idiomas.get('nivel_espanhol'),
		'outro_idioma': formacao_e_idiomas.get('outro_idioma'),
		'cv_pt': applicants.get(id).get("cv_pt"),
	}
	rows.append(aux)

applicants_df = pd.DataFrame(data=rows, columns=['id', 'pcd', 'endereco', 'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos','certificacoes', 'outras_certificacoes', 'nivel_profissional','nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'cv_pt'])

In [ ]:
with open(Data_path + 'vagas.json', 'r', encoding='utf-8') as file:
	vagas = json.load(file)

rows = []
for vaga in vagas:
	id = vaga
	perfil_vaga = vagas.get(id).get("perfil_vaga")
	aux = {
		'id': int(id),
		'estado': perfil_vaga.get('estado'),
		'vaga_especifica_para_pcd': perfil_vaga.get('vaga_especifica_para_pcd'),
		'nivel_profissional': perfil_vaga.get('nivel profissional'),
		'nivel_academico': perfil_vaga.get('nivel_academico'),
		'nivel_ingles': perfil_vaga.get('nivel_ingles'),
		'nivel_espanhol': perfil_vaga.get('nivel_espanhol'),
		'outro_idioma': perfil_vaga.get('outro_idioma'),
		'areas_atuacao': perfil_vaga.get('areas_atuacao'),
		'principais_atividades': perfil_vaga.get('principais_atividades'),
		'demais_observacoes': perfil_vaga.get('demais_observacoes'),
		'viagens_requeridas': perfil_vaga.get('viagens_requeridas')
	}
	rows.append(aux)

vagas_df = pd.DataFrame(data=rows, columns=['id', 'estado', 'vaga_especifica_para_pcd', 'nivel_profissional', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'areas_atuacao', 'principais_atividades', 'demais_observacoes', 'viagens_requeridas'])

In [ ]:
with open(Data_path + 'prospects.json', 'r', encoding='utf-8') as file:
	prospects = json.load(file)

rows = []
for prospect in prospects:
	id = prospect
	candidates = prospects.get(id).get("prospects")
	for candidate in candidates:
		aux_id:int = int(id)
		aux = {
			'id_vaga': aux_id,
			'id_applicant': int(candidate.get("codigo")),
			'situacao_candidado': candidate.get("situacao_candidado"),
			'comentario': candidate.get("comentario")
		}
		rows.append(aux)

prospects_df = pd.DataFrame(data=rows, columns=['id_vaga','id_applicant', 'situacao_candidado', 'comentario'])

In [ ]:
aux_merge_ap = pd.merge(applicants_df, prospects_df, left_on='id', right_on='id_applicant', how='inner')
merge = pd.merge(aux_merge_ap, vagas_df, left_on='id_vaga', right_on='id', how='inner', suffixes=('', '_vaga'))

df = merge[[
    'pcd',
    'endereco',
    'titulo_profissional',
    'area_atuacao',
    'conhecimentos_tecnicos',
    'certificacoes',
    'outras_certificacoes',
    'nivel_profissional',
    'nivel_academico',
    'nivel_ingles',
    'nivel_espanhol',
    'outro_idioma',
    'cv_pt',
    'situacao_candidado',
    'comentario',
    'vaga_especifica_para_pcd',
    'nivel_profissional_vaga',
    'nivel_academico_vaga',
    'nivel_ingles_vaga',
    'nivel_espanhol_vaga',
    'outro_idioma_vaga',
    'areas_atuacao',
    'principais_atividades',
    'demais_observacoes',
    'viagens_requeridas',
    'estado'
]]

df = df.drop_duplicates()

In [ ]:
del applicants
del vagas
del prospects
del applicants_df
del prospects_df
del vagas_df
del aux_merge_ap
del merge
gc.collect()

## Data Pre Processor

In [ ]:
df['pcd'] = df['pcd'].apply(lambda x: 1 if x == 'Sim' else 0)
df['vaga_especifica_para_pcd'] = df['vaga_especifica_para_pcd'].apply(lambda x: 1 if x == 'Sim' else 0)
df['pcd_match'] = ((df['pcd'] == 1) & (df['vaga_especifica_para_pcd'] == 1)).astype(int)

In [ ]:
def string_nomalizer(text):
	return ''.join(c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)).lower()

In [ ]:
df['endereco'] = df['endereco'].apply(string_nomalizer)
df['estado'] = df['estado'].apply(string_nomalizer)
df['state_match'] = ((df['endereco'] == df['estado']) & (df['endereco'] != "") & (df['estado'] != "")).astype(int)

In [ ]:
print(df.shape)
print(df.info())

In [ ]:
df.head()

## Model

## Train

## Predict

## Metrics